In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/seml/database.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
collection_name = 'week8_cora_full'
collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

Number of finished experiments : 16


In [3]:
def get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True):
    exs = [
        r for r in experiments if r['config']['model']['residual'] == residual and r['config']['model']['use_spectral_norm'] == spectral_norm and r['config']['data']['train_labels_remove_other'] == train_labels_remove_other
    ]
    return exs[0]

In [4]:
experiment = get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True)

In [5]:
print([k for k in experiment['result'].keys() if 'acc' in k])

['val_accuracy-val-train-labels-0', 'val_accuracy-val-reduced-0', 'accuracy_val-reduced', 'accuracy_id_val-reduced', 'accuracy_ood_val-reduced', 'accuracy_val-reduced_no-edges', 'accuracy_id_val-reduced_no-edges', 'accuracy_ood_val-reduced_no-edges', 'accuracy_val_loc', 'accuracy_id_val_loc', 'accuracy_ood_val_loc', 'accuracy_val_loc-no-edges', 'accuracy_id_val_loc-no-edges', 'accuracy_ood_val_loc-no-edges', 'accuracy_val-reduced-bernoulli_bernoulli', 'accuracy_id_val-reduced-bernoulli_bernoulli', 'accuracy_ood_val-reduced-bernoulli_bernoulli', 'accuracy_val-reduced-bernoulli_bernoulli-no-edges', 'accuracy_id_val-reduced-bernoulli_bernoulli-no-edges', 'accuracy_ood_val-reduced-bernoulli_bernoulli-no-edges', 'accuracy_val-reduced-normal_normal', 'accuracy_id_val-reduced-normal_normal', 'accuracy_ood_val-reduced-normal_normal', 'accuracy_val-reduced-normal_normal-no-edges', 'accuracy_id_val-reduced-normal_normal-no-edges', 'accuracy_ood_val-reduced-normal_normal-no-edges']


In [39]:

d = {}
for experiment in experiments:
    residual = experiment['config']['model']['residual']
    spectral_norm = experiment['config']['model']['use_spectral_norm']
    remove_other = experiment['config']['data']['train_labels_remove_other']
    if any('Operating_Systems' in label for label in experiment['config']['data']['val_labels']):
        ood = 'os'
    else:
        ood = 'ai'
    
    prefix = (ood, remove_other, residual, spectral_norm)
    
    for k, v in experiment['result'].items():
        mean, std = np.array(v).mean(), np.array(v).std()
        med = np.median(np.array(v))
        if 'auroc' in k:
            if 'no-edges' in k:
                no_edges = True
                k = k.replace('_no-edges', '')
                k = k.replace('-no-edges', '')
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            k, ood_type = '_'.join(k_tokens[:-1]), k_tokens[-1]
            prefix_edges = prefix + (no_edges, ood_type,)
                
            if not 'no' in k and ':' in k:
                continue
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'mean')] = [mean]
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'std')] = [std]
            #d[prefix_edges + ('feature density', k.replace('auroc:', ''), 'median')] = [med]
        elif k.startswith('accuracy'):
            k_tokens = k.split('_')
            if k_tokens[1] in ('id', 'ood'):
                k_tokens = ['_'.join(k_tokens[:2])] + k_tokens[2:]
            if 'no-edges' in k_tokens[-1]:
                k_tokens[-1] = k_tokens[-1].replace('no-edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            k_tokens = [token for token in k_tokens if len(token) > 0]
            if len(k_tokens) == 3:
                ood_type = k_tokens[-1]
                k_tokens = k_tokens[:-1]
            else:
                #ood_type = ''
                continue
            name, dataset = k_tokens
            
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'mean')] = [mean]
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'std')] = [std]
            #d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'med')] = [med]
        elif k.startswith('ece') and False:
            if 'no_edges' in k:
                k = k.replace('no_edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            d[prefix + (no_edges, '', 'ece', '', 'mean')] = [mean]
            d[prefix + (no_edges, '', 'ece', '', 'std')] = [std]
            
            #print(name, dataset, prefix + (no_edges, ood_type,))
            
            
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'mean')] = [mean]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'std')] = [std]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'median')] = [med]
                
df = pd.DataFrame(d).T.sort_index()

In [40]:
df

0
ai False False False False bernoulli AUC-ROC  7 mog:no     mean  0.48
                                                           std   0.08
                                              gpc full:no  mean  0.48
                                                           std   0.09
                                              logit energy mean  0.59
...                                                               ...
os True  True  True  True  normal    accuracy              std   0.04
                                              id           mean  0.77
                                                           std   0.04
                                              ood          mean  0.13
                                                           std   0.08

[1536 rows x 1 columns]

In [41]:
df = df.reset_index((0, 1, 2, 3, 5, -1))

In [42]:
df.shape

(1536, 7)

In [43]:
df

level_0  level_1  level_2  level_3    level_5  \
False AUC-ROC  7 mog:no          ai    False    False    False  bernoulli   
               7 mog:no          ai    False    False    False  bernoulli   
               gpc full:no       ai    False    False    False  bernoulli   
               gpc full:no       ai    False    False    False  bernoulli   
               logit energy      ai    False    False    False  bernoulli   
...                             ...      ...      ...      ...        ...   
True  accuracy                   os     True     True     True     normal   
               id                os     True     True     True     normal   
               id                os     True     True     True     normal   
               ood               os     True     True     True     normal   
               ood               os     True     True     True     normal   

                            level_8     0  
False AUC-ROC  7 mog:no        mean  0.48  
               7 mog:no         std  0.08  
               gpc full:no     mean  0.48  
               gpc full:no      std  0.09  
               logit energy    mean  0.59  
...                             ...   ...  
True  accuracy                  std  0.04  
               id              mean  0.77  
               id               std  0.04  
               ood             mean  0.13  
               ood              std  0.08  

[1536 rows x 7 columns]

In [44]:
subdfs, names = [], []
for n, g in df.groupby(['level_0', 'level_1', 'level_2', 'level_3', 'level_5', 'level_8']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)

In [45]:
df_cat = pd.concat(subdfs, axis=1)
idx = pd.MultiIndex.from_tuples(df_cat.columns, names=('OOD Classes', 'Remove OOD', 'Residual', 'Spectral Norm', 'OOD Experiment', 'Stat'))
df_cat.columns = idx
df_cat.index.names = ('Remove-Edges', '', '')
pd.set_option("display.precision", 2)
df_cat = df_cat.T.sort_index().T

In [46]:
exp_sizes = [
    1, # ood data
    1, # remove ood vertices from train
    2, # residual,
    2, # Spectral norm,
    3, # ood type
    2, # stats
]
exp_size = reduce(lambda x, y: x * y, exp_sizes)
exp_size

24

In [47]:


df_cat.iloc[:, 0 * exp_size : 1 * exp_size : 2]

OOD Classes                                           ai               \
Remove OOD                                         False                
Residual                                           False                
Spectral Norm                                      False                
OOD Experiment                                 bernoulli   loc normal   
Stat                                                mean  mean   mean   
Remove-Edges                                                            
False        AUC-ROC  7 mog:no                  4.81e-01  0.71   1.00   
                      gpc full:no               4.84e-01  0.73   1.00   
                      logit energy              5.86e-01  0.82   0.21   
                      max score                 5.73e-01  0.83   0.40   
                      total predictive entropy  5.82e-01  0.83   0.35   
             accuracy                           8.00e-01  0.80   0.74   
                      id                        8.08e-01  0.78   0.79   
                      ood                       7.31e-01   NaN   0.32   
True         AUC-ROC  7 mog:no                  1.29e-03  0.49   1.00   
                      gpc full:no               2.95e-03  0.51   1.00   
                      logit energy              9.97e-01  0.79   0.00   
                      max score                 1.00e+00  0.78   0.08   
                      total predictive entropy  1.00e+00  0.79   0.07   
             accuracy                           7.22e-01  0.78   0.71   
                      id                        7.84e-01  0.74   0.78   
                      ood                       1.60e-01   NaN   0.12   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                  4.19e-01  0.59  9.98e-01   
                      gpc full:no               4.79e-01  0.76  9.98e-01   
                      logit energy              5.72e-01  0.87  2.79e-01   
                      max score                 5.69e-01  0.88  3.77e-01   
                      total predictive entropy  5.72e-01  0.87  2.74e-01   
             accuracy                           8.06e-01  0.81  7.66e-01   
                      id                        8.13e-01  0.79  8.02e-01   
                      ood                       7.43e-01   NaN  4.40e-01   
True         AUC-ROC  7 mog:no                  0.00e+00  0.29  1.00e+00   
                      gpc full:no               8.30e-03  0.42  1.00e+00   
                      logit energy              9.91e-01  0.87  1.13e-03   
                      max score                 9.93e-01  0.86  3.43e-02   
                      total predictive entropy  9.94e-01  0.88  3.47e-03   
             accuracy                           7.29e-01  0.79  7.23e-01   
                      id                        7.92e-01  0.77  7.86e-01   
                      ood                       1.66e-01   NaN  1.51e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no         

In [49]:

df_cat.iloc[:, 1 * exp_size : 2 * exp_size : 2]

OOD Classes                                           ai                  \
Remove OOD                                          True                   
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                  4.62e-01  0.79  9.97e-01   
                      gpc full:no               4.72e-01  0.84  9.97e-01   
                      logit energy              6.00e-01  0.90  2.33e-01   
                      max score                 5.89e-01  0.90  4.72e-01   
                      total predictive entropy  5.94e-01  0.90  4.22e-01   
             accuracy                           8.27e-01  0.82  7.65e-01   
                      id                        8.32e-01  0.85  8.14e-01   
                      ood                       7.89e-01   NaN  3.20e-01   
True         AUC-ROC  7 mog:no                  4.58e-03  0.48  1.00e+00   
                      gpc full:no               4.58e-03  0.53  1.00e+00   
                      logit energy              9.96e-01  0.88  2.49e-04   
                      max score                 1.00e+00  0.85  9.73e-02   
                      total predictive entropy  1.00e+00  0.87  8.68e-02   
             accuracy                           7.23e-01  0.79  7.19e-01   
                      id                        7.87e-01  0.77  7.85e-01   
                      ood                       1.49e-01   NaN  1.31e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                  4.15e-01  0.73  9.97e-01   
                      gpc full:no               4.79e-01  0.89  9.97e-01   
                      logit energy              5.95e-01  0.87  2.79e-01   
                      max score                 5.94e-01  0.90  4.16e-01   
                      total predictive entropy  5.95e-01  0.89  2.97e-01   
             accuracy                           8.40e-01  0.83  7.96e-01   
                      id                        8.42e-01  0.86  8.30e-01   
                      ood                       8.17e-01   NaN  4.89e-01   
True         AUC-ROC  7 mog:no                  9.75e-04  0.35  1.00e+00   
                      gpc full:no               2.53e-02  0.52  1.00e+00   
                      logit energy              9.83e-01  0.90  2.27e-03   
                      max score                 9.90e-01  0.90  5.33e-02   
                      total predictive entropy  9.93e-01  0.91  9.55e-03   
             accuracy                           7.17e-01  0.78  7.21e-01   
                      id                        7.82e-01  0.76  7.82e-01   
                      ood                       1.34e-01   NaN  1.74e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                  

In [50]:

df_cat.iloc[:, 2 * exp_size : 3 * exp_size : 2]

OOD Classes                                           os               \
Remove OOD                                         False                
Residual                                           False                
Spectral Norm                                      False                
OOD Experiment                                 bernoulli   loc normal   
Stat                                                mean  mean   mean   
Remove-Edges                                                            
False        AUC-ROC  7 mog:no                  4.81e-01  0.74   1.00   
                      gpc full:no               4.84e-01  0.77   1.00   
                      logit energy              5.86e-01  0.92   0.21   
                      max score                 5.73e-01  0.91   0.40   
                      total predictive entropy  5.82e-01  0.92   0.35   
             accuracy                           8.00e-01  0.80   0.74   
                      id                        8.08e-01  0.78   0.79   
                      ood                       7.31e-01   NaN   0.32   
True         AUC-ROC  7 mog:no                  1.29e-03  0.51   1.00   
                      gpc full:no               2.95e-03  0.54   1.00   
                      logit energy              9.97e-01  0.86   0.00   
                      max score                 1.00e+00  0.83   0.08   
                      total predictive entropy  1.00e+00  0.85   0.07   
             accuracy                           7.22e-01  0.78   0.71   
                      id                        7.84e-01  0.74   0.78   
                      ood                       1.60e-01   NaN   0.12   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                  4.18e-01  0.57  9.98e-01   
                      gpc full:no               4.76e-01  0.78  9.98e-01   
                      logit energy              5.75e-01  0.94  2.79e-01   
                      max score                 5.70e-01  0.96  3.77e-01   
                      total predictive entropy  5.73e-01  0.95  2.75e-01   
             accuracy                           8.05e-01  0.81  7.64e-01   
                      id                        8.11e-01  0.79  8.00e-01   
                      ood                       7.49e-01   NaN  4.46e-01   
True         AUC-ROC  7 mog:no                  0.00e+00  0.28  1.00e+00   
                      gpc full:no               9.89e-03  0.42  1.00e+00   
                      logit energy              9.90e-01  0.94  1.68e-03   
                      max score                 9.91e-01  0.93  3.28e-02   
                      total predictive entropy  9.93e-01  0.94  3.08e-03   
             accuracy                           7.27e-01  0.78  7.22e-01   
                      id                        7.90e-01  0.78  7.85e-01   
                      ood                       1.54e-01   NaN  1.57e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no         

In [53]:

df_cat.iloc[:, 3 * exp_size : 4 * exp_size : 2]

OOD Classes                                           os                  \
Remove OOD                                          True                   
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                  4.62e-01  0.82  9.97e-01   
                      gpc full:no               4.72e-01  0.85  9.97e-01   
                      logit energy              6.00e-01  0.92  2.33e-01   
                      max score                 5.89e-01  0.93  4.72e-01   
                      total predictive entropy  5.94e-01  0.93  4.22e-01   
             accuracy                           8.27e-01  0.82  7.65e-01   
                      id                        8.32e-01  0.85  8.14e-01   
                      ood                       7.89e-01   NaN  3.20e-01   
True         AUC-ROC  7 mog:no                  4.58e-03  0.51  1.00e+00   
                      gpc full:no               4.63e-03  0.57  1.00e+00   
                      logit energy              9.96e-01  0.90  2.49e-04   
                      max score                 1.00e+00  0.89  9.72e-02   
                      total predictive entropy  1.00e+00  0.90  8.68e-02   
             accuracy                           7.23e-01  0.79  7.19e-01   
                      id                        7.87e-01  0.77  7.85e-01   
                      ood                       1.49e-01   NaN  1.31e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  7 mog:no                      0.42  0.72  9.97e-01   
                      gpc full:no                   0.48  0.88  9.97e-01   
                      logit energy                  0.60  0.87  2.76e-01   
                      max score                     0.60  0.93  4.15e-01   
                      total predictive entropy      0.60  0.90  2.94e-01   
             accuracy                               0.84  0.84  7.97e-01   
                      id                            0.85  0.86  8.32e-01   
                      ood                           0.81   NaN  4.83e-01   
True         AUC-ROC  7 mog:no                      0.00  0.35  1.00e+00   
                      gpc full:no                   0.02  0.52  1.00e+00   
                      logit energy                  0.98  0.93  1.50e-03   
                      max score                     0.99  0.94  4.97e-02   
                      total predictive entropy      0.99  0.94  8.05e-03   
             accuracy                               0.72  0.78  7.22e-01   
                      id                            0.78  0.76  7.83e-01   
                      ood                           0.13   NaN  1.71e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                  